In [34]:
import tensorflow as tf

In [35]:
from tensorflow import keras as kr

In [36]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [37]:
from keras.preprocessing.sequence import pad_sequences

In [38]:
import pandas as pd

In [39]:
data=pd.read_csv('sentiment\IMDB Dataset.csv')

In [40]:
data.head(4)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative


In [41]:
import spacy
import nltk
from nltk import wordpunct_tokenize
nlp = spacy.load("en_core_web_sm")
st=nlp.Defaults.stop_words
import re
import string
e=string.punctuation
def text_pro(text):
    pattern=re.compile(r'<.*?>') #remove html tag
    text=re.sub(pattern,'',text)
    pattern=re.compile('https?\S*') #remove url
    text=re.sub(pattern,'',text)
    for i in e:                   #remove puntuation
      text=text.replace(i,'')
    text=text.lower() #make lower
    text=wordpunct_tokenize(text) #tokenizatio
    list=[]
    for i in text:        # collect only alphabetic
        if i.isalpha():
           list.append(i)
    text=list[:]
    text=[ i for i in text if i not in st if len(i) >= 2] #stop words
    text=" ".join(text)
    doc = nlp(text)
    text= [token.lemma_ for token in doc] #lematiztion
    text=" ".join(text)
    list.clear()
    return text

In [42]:
data['transform']=data['review'].apply(text_pro)

In [48]:
data.head()

,review,sentiment,transform,token
0,One of the other reviewers has mentioned that ...,positive,reviewer mention watch oz episode you ll hook ...,"[reviewer, mention, watch, oz, episode, you, l..."
1,A wonderful little production. <br /><br />The...,positive,wonderful little production filming technique ...,"[wonderful, little, production, filming, techn..."
2,I thought this was a wonderful way to spend ti...,positive,think wonderful way spend time hot summer week...,"[think, wonderful, way, spend, time, hot, summ..."
3,Basically there's a family where a little boy ...,negative,basically there s family little boy jake think...,"[basically, there, s, family, little, boy, jak..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter matteis love time money visually stunni...,"[petter, matteis, love, time, money, visually,..."


In [49]:
y=data.drop(['transform','review'],axis=1)

In [50]:
data['token']=data['transform'].apply(lambda x: x.split())

In [51]:
data.head()

,review,sentiment,transform,token
0,One of the other reviewers has mentioned that ...,positive,reviewer mention watch oz episode you ll hook ...,"[reviewer, mention, watch, oz, episode, you, l..."
1,A wonderful little production. <br /><br />The...,positive,wonderful little production filming technique ...,"[wonderful, little, production, filming, techn..."
2,I thought this was a wonderful way to spend ti...,positive,think wonderful way spend time hot summer week...,"[think, wonderful, way, spend, time, hot, summ..."
3,Basically there's a family where a little boy ...,negative,basically there s family little boy jake think...,"[basically, there, s, family, little, boy, jak..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter matteis love time money visually stunni...,"[petter, matteis, love, time, money, visually,..."


In [ ]:
import gensim      #we used it to extract weights which has been used in weight init 
from gensim.models import Word2Vec

In [ ]:
w2vmodel = Word2Vec(sentences=data['token'], vector_size=100, window=5, min_count=2,sg=0)  #sg=1 means skip-gram
len(w2vmodel.wv.key_to_index)

In [ ]:
w2vmodel.build_vocab(['<nothing>'], update=True) #update new word in vocab

In [ ]:
vocab_w2v=w2vmodel.wv.key_to_index

In [ ]:
w2vmodel.wv.most_similar('movie')

[('film', 0.804800271987915),
 ('moviesbut', 0.7931540012359619),
 ('therefor', 0.7757278680801392),
 ('moviebut', 0.773337185382843),
 ('watchin', 0.7680141925811768),
 ('moviejust', 0.7637479305267334),
 ('movieoverall', 0.7616720795631409),
 ('moviedont', 0.7604532241821289),
 ('movieit', 0.7597306966781616),
 ('havnt', 0.7595797777175903)]

In [325]:
token=Tokenizer(oov_token='<nothing>',lower=False,num_words=150000)

In [326]:
token.fit_on_texts(data['transform'])

In [259]:
import pickle
with open("sentiment_token.pkl", "wb") as file:
    pickle.dump(token, file)

In [327]:
input=len(token.word_index)+1
input

197247

In [328]:
vocab=token.word_index

In [329]:
df=token.texts_to_sequences(data['transform'])

In [330]:
data['vc']=df

In [331]:
data.head()

,review,sentiment,transform,token,vc
0,One of the other reviewers has mentioned that ...,positive,reviewer mention watch oz episode you ll hook ...,"[reviewer, mention, watch, oz, episode, you, l...","[882, 268, 10, 2704, 137, 56, 240, 1980, 87, 4..."
1,A wonderful little production. <br /><br />The...,positive,wonderful little production filming technique ...,"[wonderful, little, production, filming, techn...","[250, 41, 201, 2139, 1406, 69614, 69615, 1148,..."
2,I thought this was a wonderful way to spend ti...,positive,think wonderful way spend time hot summer week...,"[think, wonderful, way, spend, time, hot, summ...","[15, 250, 24, 304, 8, 643, 1129, 1922, 269, 55..."
3,Basically there's a family where a little boy ...,negative,basically there s family little boy jake think...,"[basically, there, s, family, little, boy, jak...","[502, 107, 11, 102, 41, 163, 2689, 15, 107, 11..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter matteis love time money visually stunni...,"[petter, matteis, love, time, money, visually,...","[69618, 26918, 23, 8, 155, 1719, 1116, 3, 10, ..."


In [332]:
pad=pad_sequences(data['vc'],padding='post',maxlen=500)

In [333]:
len(token.word_index)+1

197247

In [334]:
def embedding_matrix(model):
    embedding_matrix = np.zeros((197915,300))
    for word, i in vocab_w2v.items():
        if word in model.wv:
            embedding_matrix[i] = model.wv[word]
    return embedding_matrix

In [ ]:
t=embedding_matrix(w2vmodel)

In [ ]:
t.shape

(197915, 300)

In [ ]:
weights=pd.DataFrame(t)

In [ ]:
weights.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.123084,0.229055,0.044114,-0.022289,0.025585,-0.397156,0.207203,0.228648,-0.133080,0.009408,...,-0.101809,0.285936,-0.018794,0.098040,0.263403,0.173988,0.031822,-0.102886,0.107092,0.040532
1,0.075653,0.091749,0.068950,-0.088446,-0.010065,-0.304310,0.090545,0.140416,-0.074545,0.008355,...,0.039410,0.259521,0.023549,0.080092,0.266669,0.193880,-0.070600,-0.113709,0.047872,0.019838
2,-0.149682,0.129270,0.058180,0.275058,0.186702,-0.176132,0.274740,0.432175,-0.115903,0.039725,...,0.127315,0.055274,0.189023,0.053411,0.331532,0.037454,-0.008474,0.053535,0.427821,-0.092465
3,-0.085838,0.180012,0.185422,-0.098587,0.089854,-0.180285,0.107858,0.171316,-0.024090,-0.165032,...,-0.086851,0.077646,0.180063,0.136643,0.196446,-0.119623,0.115355,0.056706,-0.017874,-0.095986
4,0.156592,0.054427,0.081971,-0.149997,-0.021473,-0.013660,0.076491,0.285821,-0.058431,-0.018980,...,0.028476,0.352595,0.169160,0.091444,0.309627,-0.065617,0.086609,-0.022969,0.090900,0.032102


In [78]:
import pickle

In [79]:
with open("sentiment_token.pkl", "wb") as file:
    pickle.dump(token, file)

In [80]:
from sklearn.preprocessing import LabelEncoder

In [81]:
encoder = LabelEncoder()
y=encoder.fit_transform(y['sentiment'])

In [82]:
from sklearn.model_selection import train_test_split

In [335]:
x_train,x_test,y_train,y_test=train_test_split(pad,y,random_state=121,test_size=.2)

In [336]:
x_train.shape

(40000, 500)

In [337]:
x_test.shape

(10000, 500)

In [338]:
y_test.shape

(10000,)

In [339]:
y_train.shape

(40000,)

In [340]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout,Embedding,Flatten,GRU,Bidirectional
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [341]:
len(x_train[0])

500

In [342]:
from tensorflow.keras.activations import swish

In [344]:
model=Sequential()
model.add(Embedding(input_dim=input,output_dim=200,trainable=True))
model.add(Bidirectional(LSTM(128,return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(128,return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(64,return_sequences=False)))
model.add(Dropout(0.2))
model.add(Dense(64,activation='swish'))
#model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))

In [345]:
optimizer=Adam(learning_rate=0.001)

In [346]:
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [347]:
history = model.fit(
        x_train, y_train,
        validation_data=(x_test, y_test),
        epochs=3,  # Set to a low number for quick experimentation
        batch_size=32,
        verbose=1)

Epoch 1/3
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1697s 1s/step - accuracy: 0.6884 - loss: 0.5774 - val_accuracy: 0.8515 - val_loss: 0.3636
Epoch 2/3
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1873s 1s/step - accuracy: 0.8514 - loss: 0.3625 - val_accuracy: 0.8778 - val_loss: 0.2951
Epoch 3/3
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1744s 1s/step - accuracy: 0.9310 - loss: 0.1895 - val_accuracy: 0.8807 - val_loss: 0.3151


In [287]:
model.save('lstm_model.h5')

In [ ]:
model.summary()

In [58]:
from keras.activations import leaky_relu,elu

In [ ]:
model=Sequential()
model.add(Embedding(input_dim=input,output_dim=300,weights=[t],trainable=False))
model.add(Bidirectional(GRU(128,return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(GRU(64,return_sequences=False)))
model.add(Dropout(0.4))
model.add(Dense(32,activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))

In [67]:
optimizer=Adam(learning_rate=0.001)

In [68]:
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

In [69]:
history = model.fit(
        x_train, y_train,
        validation_data=(x_test, y_test),
        epochs=5,# Set to a low number for quick experimentation
        batch_size=32,
        verbose=1)

Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 74s 56ms/step - accuracy: 0.5989 - loss: 0.6572 - val_accuracy: 0.7964 - val_loss: 0.4410
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 77s 52ms/step - accuracy: 0.7995 - loss: 0.4309 - val_accuracy: 0.8167 - val_loss: 0.3959
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 82s 52ms/step - accuracy: 0.8460 - loss: 0.3501 - val_accuracy: 0.8519 - val_loss: 0.3486
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 81s 51ms/step - accuracy: 0.8709 - loss: 0.2998 - val_accuracy: 0.8565 - val_loss: 0.3399
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 82s 51ms/step - accuracy: 0.8981 - loss: 0.2526 - val_accuracy: 0.8520 - val_loss: 0.3648


In [70]:
model.save('gru_model.h5')

In [7]:
from keras.models import load_model

In [24]:
model=load_model('gru_model.h5')

In [288]:
model=load_model('lstm_model.h5')
model.compile(optimizer=optimizer,loss='binary_crossentropy', metrics=['accuracy'])
model.evaluate(x_test, y_test, verbose=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 36s 112ms/step - accuracy: 0.8919 - loss: 0.2979


[0.2971433103084564, 0.8881000280380249]

In [ ]:
import pickle
with open ('sentiment_token.pkl','rb') as file:
  token=pickle.load(file)

In [368]:
text='''No one expects the Star Trek movies to be high art, but the fans do expect a movie that is as good as some of the best episodes. Unfortunately, this movie had a muddled, implausible plot that just left me cringing - this is by far the worst of the nine (so far) movies. Even the chance to watch the well known characters interact in another movie can't save this movie - including the goofy scenes with Kirk, Spock and McCoy at Yosemite.<br /><br />I would say this movie is not worth a rental, and hardly worth watching, however for the True Fan who needs to see all the movies, renting this movie is about the only way you'll see it - even the cable channels avoid this movie.'''

In [369]:
text=text_pro(text)
text

'expect star trek movie high art fan expect movie good good episode unfortunately movie muddle implausible plot leave cringe far bad far movie chance watch know character interact movie can not save movie include goofy scenes kirk spock mccoy yosemitei movie worth rental hardly worth watch true fan need movie rent movie way you ll cable channel avoid movie'

In [370]:
df=token.texts_to_sequences(text)
df[0]

[33876]

In [371]:
pad=pad_sequences(df,padding='post',maxlen=500)
pad

array([[33876,     0,     0, ...,     0,     0,     0],
       [    1,     0,     0, ...,     0,     0,     0],
       [21603,     0,     0, ...,     0,     0,     0],
       ...,
       [    1,     0,     0, ...,     0,     0,     0],
       [  386,     0,     0, ...,     0,     0,     0],
       [33876,     0,     0, ...,     0,     0,     0]], dtype=int32)

In [372]:
float(model.predict(pad)[0][0])

12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 221ms/step


0.4439902901649475

In [353]:
from sklearn.metrics import roc_auc_score,roc_curve,accuracy_score,auc,precision_score

In [ ]:
p=model.predict(x_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 86s 275ms/step


In [ ]:
fpr,tpr,thr=roc_curve(y_test,p)

In [356]:
import numpy as np

In [ ]:
accuracy=[]
for i in thr:
  y_prd=np.where(p>i,1,0)
  x=accuracy_score(y_test,y_prd)
  accuracy.append(x)

In [358]:
d1=pd.DataFrame(thr,columns=["thresold"])
d2=pd.DataFrame(accuracy,columns=["accuracy_score"])
d=pd.concat([d1,d2],axis=1)

In [359]:
d.sort_values(by='accuracy_score',ascending=False).head(2)

,thresold,accuracy_score
1023,0.566981,0.8839
1026,0.565401,0.8839


In [360]:
p=model.predict(pad)[0][0]
if p>=.566981:
  print('+ve' ,p)
else:
  print('-ve',(1-p))

12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 132ms/step
-ve 0.5560097


In [383]:
import flask
from flask import Flask,request,render_template
app=Flask(__name__,template_folder="template")
@app.route('/')
def home_page():
    return render_template("index.html")
@app.route('/predict',methods=["POST"])
def pred_page():
    data=str(request.form.get('review'))
    text=text_pro(data)
    df=token.texts_to_sequences(text)
    pad=pad_sequences(df,maxlen=500,padding='post')
    p=model.predict(pad)[0][0]
    if p>=0.5:
       x= f'posetive {p*100: .2f}%'
    else:
       p=(1-p)
       x= f'negetive  {p*100: .2f}%'
    return render_template("result.html",x=x)

if __name__=="__main__":
 app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit


12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 261ms/step


127.0.0.1 - - [27/Nov/2024 16:55:45] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Nov/2024 16:55:45] "POST /predict HTTP/1.1" 200 -
